In [36]:
from kafka import KafkaProducer
import pandas as pd
from pytz import timezone
import csv
from time import sleep
from json import dumps
import datetime as dt
import random

In [37]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer


def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
#         print('Message published successfully. Data: ' )
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

def rand_sample(a,b):
    return random.randrange(a,b)      

In [38]:
if __name__ == '__main__':
    
    df_memory = pd.read_csv('Streaming_Linux_memory.csv')
    
    producer = connect_kafka_producer()
    
    topic = 'memory'
    
    Initial = {4:0,5:0,6:0,7:0,8:0}
    Start = {4:0,5:0,6:0,7:0,8:0}    
    end =   {4:0,5:0,6:0,7:0,8:0}
    limit = {4:0,5:0,6:0,7:0,8:0}
    
    for value in df_memory['machine'].unique():
        temp = df_memory[df_memory['machine']==int(value) ].copy()
        Initial[value]=temp.index[0]
        Start[value]=temp.index[0]
        limit[value] =temp.index[-1]
    
    df_y = pd.DataFrame()
    while True:
        
        df_x = pd.DataFrame()
        temp = pd.DataFrame()
        Temp_y = pd.DataFrame()
        for value in df_memory['machine'].unique():
            skip = False
            ran = rand_sample(20,80) 
            rn = rand_sample(0,5) 
            end[value] = Start[value]+ ran + rn
            
            if (int(end[value]) >= int(limit[value])):
                m_end = end[value]-ran-rn
                a = abs(limit[value]-m_end)
                b = abs(end[value]-limit[value])
                
                temp = temp.append(df_memory.loc[m_end:limit[value]].copy())
                temp = temp.append(df_memory.loc[Initial[value]:Initial[value]+b].copy())
                end[value] = Initial[value]+b+1

                temp = temp.reset_index()
                df_x = df_x.append(temp.loc[:ran])
                Temp_y = df_y.append(temp.loc[ran+1:])
                skip = True
            
            if skip == False: 
                df_x = df_x.append(df_memory.loc[Start[value]:Start[value]+ran+1,].copy())  
                Temp_y = Temp_y.append(df_memory.loc[end[value]-rn:end[value],].copy())
                        
            Start[value] = end[value]
        
        ts = str(dt.datetime.now(timezone('UTC')).timestamp())[0:10]
        df_x['ts'] = ts
        Temp_y['ts'] = ts
 
        for col in df_x.columns:
            df_x[col] = df_x[col].astype('str')
            
        for col in Temp_y.columns:
            Temp_y[col] = Temp_y[col].astype('str')
   
        list_x = []
        list_y =[]
        
        for ind in df_x.index:
            list_x.append(dict(df_x.loc[ind]))
       
        for ind in df_y.index:
            list_y.append(dict(df_y.loc[ind]))

        publish_message(producer_instance=producer,topic_name=topic, data=(list_y+list_x))
        
        df_y = Temp_y.copy()
        sleep(10)

Exception in publishing message.
KafkaTimeoutError: Failed to update metadata after 60.0 secs.
Exception in publishing message.
KafkaTimeoutError: Failed to update metadata after 60.0 secs.


KeyboardInterrupt: 